In [0]:
# try:
#     import dlt
# except ImportError:
#     class dlt:
#         def table(comment, **options):
#             def _(f):
#                 pass
#             return _

# import os
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, DoubleType, StringType, BooleanType, ArrayType, LongType, MapType
# from pyspark.sql.functions import from_json, col, size, explode, json_tuple, get_json_object, from_unixtime
# from pyspark.sql.types import TimestampType, DateType
# import dlt
# from pyspark.sql import functions as F

# # Define schema for the incoming JSON data
# base_schema = StructType([
#     StructField("TransactionID", StringType(), True),
#     StructField("UserID", StringType(), True),
#     StructField("TransactionDate", TimestampType(), True),
#     StructField("TransactionAmount", DoubleType(), True),
#     StructField("TransactionType", StringType(), True),
#     StructField("MerchantID", StringType(), True),
#     StructField("Currency", StringType(), True),
#     StructField("TransactionStatus", StringType(), True),
#     StructField("DeviceType", StringType(), True),
#     StructField("IP_Address", StringType(), True),
#     StructField("PaymentMethod", StringType(), True),
#     StructField("SuspiciousFlag", BooleanType(), True),
#     StructField("IsFraud", BooleanType(), True),
#     StructField("AnomalyScore", DoubleType(), True),
#     StructField("Age", IntegerType(), True),
#     StructField("Gender", StringType(), True),
#     StructField("AccountCreationDate", TimestampType(), True),
#     StructField("Location", StringType(), True),
#     StructField("LocationCoordinates", StringType(), True),
#     StructField("AccountStatus", StringType(), True),
#     StructField("DeviceID", StringType(), True),
#     StructField("UserProfileCompleteness", DoubleType(), True),
#     StructField("PreviousFraudAttempts", IntegerType(), True)
# ])

# # Create or refresh the raw (bronze) table
# @dlt.table(
#     comment="Bronze table for raw fraud detection data",
#     table_properties={
#         "quality": "bronze"
#     }
# )
# def br_fraud_detection_raw_data_historical():
#     raw_data = spark.readStream \
#         .format("cloudFiles") \
#         .option("cloudFiles.format", "json") \
#         .option("cloudFiles.inferColumnTypes", "true") \
#         .option("cloudFiles.rescuedDataColumn", "rescue_col") \
#         .load("s3://fraud-detection-raw-data1/year=*/transactions_*.json")
    
#     return raw_data

# # Create or refresh the fact table (silver quality)
# @dlt.table(
#     comment="Silver table for fraud detection transactions",
#     table_properties={
#         "quality": "silver"
#     }
# )
# def si_transactions_fact():
#     raw_data = dlt.read_stream("br_fraud_detection_raw_data_historical")
    
#     # Selecting distinct columns to create the fact table
#     return raw_data.select(
#         "TransactionID",
#         "UserID",
#         "TransactionDate",
#         "TransactionAmount",
#         "TransactionType",
#         "MerchantID",
#         "Currency",
#         "TransactionStatus",
#         "DeviceType",
#         "IP_Address",
#         "PaymentMethod",
#         "SuspiciousFlag",
#         "IsFraud",
#         "AnomalyScore",
#         "Age",
#         "Gender",
#         "AccountCreationDate",
#         "Location",
#         "LocationCoordinates",
#         "AccountStatus",
#         "DeviceID",
#         "UserProfileCompleteness",
#         "PreviousFraudAttempts"
#     ).distinct()



# # import dlt
# # from pyspark.sql.functions import first, when

# # @dlt.table(name="si_merchants_dimension", comment="Silver quality Merchants Dimension table", table_properties={"quality": "silver"})
# # def si_merchants_dimension():
# #     raw_data = dlt.read("br_fraud_detection_raw_data_historical")
# #     return raw_data.groupBy("MerchantID").agg(
# #         F.first("LocationCoordinates").alias("MerchantLocation"),
# #         # Replace 'N/A' with the actual column name or use a default value
# #         F.first(F.when(F.col("ContactColumnName").isNotNull(), F.col("ContactColumnName")).otherwise("Unknown")).alias("MerchantContact"),  # Replace 'ContactColumnName' with the actual column name
# #         F.first("Active").alias("MerchantStatus"),
# #         F.when(raw_data.MerchantID.isin("Amazon", "eBay", "Best Buy"), "Electronics")
# #             .when(raw_data.MerchantID.isin("Walmart", "Target"), "Groceries")
# #             .otherwise("Other").alias("MerchantCategory")
# #     )

# from pyspark.sql.functions import first

# @dlt.table(name="si_users_dimension", comment="Silver quality Users Dimension table", table_properties={"quality": "silver"})
# def si_users_dimension():
#     raw_data = dlt.read("br_fraud_detection_raw_data_historical")
#     return raw_data.groupBy("UserID").agg(
#         first("Age").alias("Age"),
#         first("Gender").alias("Gender"),
#         first("AccountCreationDate").alias("AccountCreationDate"),
#         first("AccountStatus").alias("AccountStatus"),
#         first("UserProfileCompleteness").alias("UserProfileCompleteness"),
#         first("PreviousFraudAttempts").alias("PreviousFraudAttempts"),
#         first("Location").alias("UserLocation")
#     )


# @dlt.table(name="si_devices_dimension", comment="Silver quality Devices Dimension table", table_properties={"quality": "silver"})
# def si_devices_dimension():
#     raw_data = dlt.read("br_fraud_detection_raw_data_historical")
#     return raw_data.groupBy("DeviceID").agg(
#         first("DeviceType").alias("DeviceType"),
#         first("IP_Address").alias("IP_Address")
#     )



# # import dlt
# # from pyspark.sql.functions import year, month, dayofweek, weekofyear, quarter, dayofweek, when

# # @dlt.table(name="si_transaction_date_dimension", comment="Silver quality Transaction Date Dimension table", table_properties={"quality": "silver"})
# # def si_transaction_date_dimension():
# #     raw_data = dlt.read("br_fraud_detection_raw_data_historical").filter("TransactionDate IS NOT NULL")
# #     return raw_data.selectExpr(
# #         "DISTINCT TransactionDate AS date_key",
# #         "year(TransactionDate) AS year",
# #         "month(TransactionDate) AS month",
# #         "day(TransactionDate) AS day",
# #         "weekofyear(TransactionDate) AS week",
# #         "quarter(TransactionDate) AS quarter",
# #         when(dayofweek("TransactionDate") == 1, 'Sunday')
# #         .when(dayofweek("TransactionDate") == 2, 'Monday')
# #         .when(dayofweek("TransactionDate") == 3, 'Tuesday')
# #         .when(dayofweek("TransactionDate") == 4, 'Wednesday')
# #         .when(dayofweek("TransactionDate") == 5, 'Thursday')
# #         .when(dayofweek("TransactionDate") == 6, 'Friday')
# #         .otherwise('Saturday').alias("day_of_week"),
# #         when(month("TransactionDate") == 1, 'January')
# #         .when(month("TransactionDate") == 2, 'February')
# #         .when(month("TransactionDate") == 3, 'March')
# #         .when(month("TransactionDate") == 4, 'April')
# #         .when(month("TransactionDate") == 5, 'May')
# #         .when(month("TransactionDate") == 6, 'June')
# #         .when(month("TransactionDate") == 7, 'July')
# #         .when(month("TransactionDate") == 8, 'August')
# #         .when(month("TransactionDate") == 9, 'September')
# #         .when(month("TransactionDate") == 10, 'October')
# #         .when(month("TransactionDate") == 11, 'November')
# #         .otherwise('December').alias("month_name")
# #     )


# # import dlt
# # from pyspark.sql.functions import sum, count

# # @dlt.table(name="go_transaction_trends", comment="Gold quality Transaction Trends Analysis table", table_properties={"quality": "gold"})
# # def go_transaction_trends():
# #     transactions_fact = dlt.read("si_transactions_fact")
# #     transaction_date_dimension = dlt.read("si_transaction_date_dimension")
# #     return transactions_fact.join(transaction_date_dimension, transactions_fact.TransactionDate == transaction_date_dimension.date_key) \
# #         .groupBy("date_key", "year", "month", "TransactionType", "IsFraud") \
# #         .agg(
# #             sum("TransactionAmount").alias("total_transaction_amount"),
# #             count("TransactionID").alias("total_transactions"),
# #             sum(when(transactions_fact.IsFraud == 1, transactions_fact.TransactionAmount).otherwise(0)).alias("fraudulent_amount"),
# #             count(when(transactions_fact.IsFraud == 1, transactions_fact.TransactionID)).alias("fraudulent_transactions")
# #         )



# # Create or Refresh Gold quality table for user behavior metrics
# @dlt.table(
#   name="go_user_behavior_metrics",
#   comment="Gold quality table for user behavior metrics",
#   table_properties={"quality": "gold"
#   }
# )
# def go_user_behavior_metrics():
#     return (
#         dlt.read("si_transactions_fact")
#         .groupBy("UserID")
#         .agg(
#             F.count("TransactionID").alias("total_transactions"),
#             F.avg("TransactionAmount").alias("avg_transaction_amount"),
#             F.count(F.when(F.col("IsFraud") == 1, "TransactionID")).alias("total_fraud_transactions"),
#             F.countDistinct("DeviceID").alias("unique_devices"),
#             F.max("TransactionDate").alias("last_transaction_date"),
#             F.min("TransactionDate").alias("first_transaction_date")
#         )
#     )



# # Create or Refresh Gold quality table for merchant risk assessment
# @dlt.table(
#   name="go_merchant_risk_assessment",
#   comment="Gold quality table for merchant risk assessment",
#   table_properties={"quality": "gold"
#   }
# )
# def go_merchant_risk_assessment():
#     return (
#         dlt.read("si_transactions_fact")
#         .groupBy("MerchantID")
#         .agg(
#             F.count("TransactionID").alias("total_transactions"),
#             F.count(F.when(F.col("IsFraud") == 1, "TransactionID")).alias("total_fraud_transactions"),
#             F.sum(F.when(F.col("IsFraud") == 1, F.col("TransactionAmount")).otherwise(0)).alias("total_fraudulent_amount"),
#             F.avg("AnomalyScore").alias("avg_anomaly_score"),
#             F.countDistinct("UserID").alias("unique_users")
#         )
#     )



# # Create or Refresh Gold quality table for real-time fraud detection
# @dlt.table(
#   name="go_real_time_fraud_detection",
#   comment="Gold quality table for real-time fraud detection",
#   table_properties={"quality": "gold"
#   }
# )
# def go_real_time_fraud_detection():
#     return (
#         dlt.read("si_transactions_fact")
#         .filter((F.col("AnomalyScore") > 0.5) | (F.col("IsFraud") == 1))
#         .select(
#             "TransactionID",
#             "TransactionDate",
#             "TransactionAmount",
#             "TransactionType",
#             "UserID",
#             "MerchantID",
#             "DeviceID",
#             "IP_Address",
#             "AnomalyScore",
#             "IsFraud",
#             F.when(F.col("AnomalyScore") > 0.8, "High Risk")
#              .when(F.col("AnomalyScore") > 0.5, "Moderate Risk")
#              .otherwise("Low Risk").alias("fraud_risk_level")
#         )
#     )




# # Create or Refresh Gold quality table for predictive model features
# @dlt.table(
#   name="go_predictive_model_features",
#   comment="Gold quality table for predictive model features",
#   table_properties={"quality": "gold"
#   }
# )
# def go_predictive_model_features():
#     # Reading the fact table and metrics from user behavior and merchant risk assessment
#     transactions_fact = dlt.read("si_transactions_fact")
#     user_behavior_metrics = dlt.read("go_user_behavior_metrics")
#     merchant_risk_assessment = dlt.read("go_merchant_risk_assessment")
    
#     # Joining tables to include user and merchant metrics for predictive features
#     return (
#         transactions_fact
#         .join(user_behavior_metrics, transactions_fact.UserID == user_behavior_metrics.UserID, "inner")
#         .join(merchant_risk_assessment, transactions_fact.MerchantID == merchant_risk_assessment.MerchantID, "inner")
#         .select(
#             transactions_fact.TransactionID,
#             transactions_fact.TransactionAmount,
#             transactions_fact.TransactionType,
#             transactions_fact.AnomalyScore,
#             user_behavior_metrics.total_transactions.alias("user_total_transactions"),
#             user_behavior_metrics.avg_transaction_amount.alias("user_avg_transaction_amount"),
#             user_behavior_metrics.total_fraud_transactions.alias("user_fraud_transactions"),
#             merchant_risk_assessment.total_fraud_transactions.alias("merchant_fraud_transactions"),
#             merchant_risk_assessment.avg_anomaly_score.alias("merchant_avg_anomaly_score"),
#             transactions_fact.IsFraud
#         )
#     )


# # Create or Refresh Gold quality table for reporting and dashboarding
# @dlt.table(
#   name="go_fraud_detection_dashboard_metrics",
#   comment="Gold quality table for reporting and dashboarding metrics",
#   table_properties={"quality": "gold"
#   }
# )
# def go_fraud_detection_dashboard_metrics():
#     # Reading the transactions fact table
#     transactions_fact = dlt.read("si_transactions_fact")
    
#     # Aggregating the required metrics for fraud detection dashboarding
#     return (
#         transactions_fact
#         .agg(
#             F.count("TransactionID").alias("total_transactions"),
#             F.sum(F.when(transactions_fact.IsFraud == 1, 1).otherwise(0)).alias("total_fraud_transactions"),
#             F.sum(F.when(transactions_fact.IsFraud == 1, transactions_fact.TransactionAmount).otherwise(0)).alias("total_fraudulent_amount"),
#             F.countDistinct("UserID").alias("unique_users"),
#             F.countDistinct("MerchantID").alias("unique_merchants"),
#             F.avg("AnomalyScore").alias("avg_anomaly_score"),
#             F.max("TransactionDate").alias("last_transaction_date")
#         )
#     )


In [0]:
%load_ext autoreload
%autoreload 2

In [ ]:
from my_project.fraud_detection_data_python import *


# from my_project import fraud_detection_data_python

# fraud_detection_data_python.br_fraud_detection_raw_data_historical()
# fraud_detection_data_python.si_transactions_fact()
# fraud_detection_data_python.si_users_dimension()
# fraud_detection_data_python.si_devices_dimension()
# fraud_detection_data_python.go_user_behavior_metrics()
# fraud_detection_data_python.go_merchant_risk_assessment()
# fraud_detection_data_python.go_real_time_fraud_detection()
# fraud_detection_data_python.go_predictive_model_features()
# # Load Gold fraud detection dashboard metrics 
# fraud_detection_data_python.go_fraud_detection_dashboard_metrics()
